# MQTT

> Setting up **MQTT** on Linux involves configuring a **broker**, optionally securing it, and connecting **clients** (like IoT devices, sensors, or Raspberry Pi apps) to publish/subscribe to topics.


- skip_showdoc: true
- skip_exec: true

## 📦 1. What is MQTT?

**MQTT (Message Queuing Telemetry Transport)** is a lightweight publish/subscribe messaging protocol designed for IoT and low-bandwidth systems.

* Broker = server (e.g., Mosquitto)
* Client = device/app that publishes or subscribes to topics
* Topic = a UTF-8 string like `sensor/temp/office`

---

## ⚙️ 2. Install an MQTT Broker (Mosquitto) on Linux

### ✅ Ubuntu / Debian

```bash
sudo apt update
sudo apt install -y mosquitto mosquitto-clients
```

### ✅ Fedora / CentOS / RHEL

```bash
sudo dnf install -y mosquitto mosquitto-clients
```

### ✅ Arch Linux

```bash
sudo pacman -S mosquitto mosquitto-clients
```

### Start & Enable:

```bash
sudo systemctl enable mosquitto
sudo systemctl start mosquitto
```

Check status:

```bash
sudo systemctl status mosquitto
```

## 🔎 3. Test MQTT Locally

In one terminal (subscriber):

```bash
mosquitto_sub -h localhost -t "test/topic"
```

In another (publisher):

```bash
mosquitto_pub -h localhost -t "test/topic" -m "Hello MQTT!"
```

You should see the message appear in the first terminal.

---

## 🔐 4. Add Authentication (Username + Password)

1. **Create password file**

```bash
sudo mosquitto_passwd -c /etc/mosquitto/passwd myuser
```

2. **Edit Mosquitto config:**
   Create or modify `/etc/mosquitto/conf.d/auth.conf`:

```ini
allow_anonymous false
password_file /etc/mosquitto/passwd
listener 1883
```

3. **Restart Mosquitto:**

```bash
sudo systemctl restart mosquitto
```

4. **Test authenticated connection:**

```bash
mosquitto_pub -h localhost -t "secure/test" -u myuser -P your_password -m "Secure test"
```

---

## 🔒 5. Enable TLS Encryption (Optional but Recommended)

### Create a self-signed cert:

```bash
mkdir ~/mosquitto_certs && cd ~/mosquitto_certs
openssl req -x509 -newkey rsa:2048 -days 365 -nodes -keyout mqtt.key -out mqtt.crt
```

### Configure TLS in `/etc/mosquitto/conf.d/tls.conf`:

```ini
listener 8883
cafile /path/to/mqtt.crt
certfile /path/to/mqtt.crt
keyfile /path/to/mqtt.key
require_certificate false
```

Restart:

```bash
sudo systemctl restart mosquitto
```

Test:

```bash
mosquitto_pub --cafile mqtt.crt -h localhost -p 8883 -t "secure/topic" -m "Encrypted"
```

---

## 🧠 6. Configure Persistence (Retain Data After Restart)

In `/etc/mosquitto/mosquitto.conf`:

```ini
persistence true
persistence_location /var/lib/mosquitto/
```

Also useful:

```ini
log_dest file /var/log/mosquitto/mosquitto.log
```

Restart the broker after changing configs.

---

## 📲 7. Connect Devices / Clients

### Python (with `paho-mqtt`):

```bash
pip install paho-mqtt
```

```python
import paho.mqtt.client as mqtt

client = mqtt.Client()
client.connect("localhost", 1883, 60)
client.publish("test/topic", "Hello from Python!")
client.loop_start()
```

### Raspberry Pi Pico W (MicroPython):

Use `umqtt.simple` or `umqtt.robust` libraries to connect and publish.

---

## 📡 8. Remote Access and Firewalls

* Open port `1883` or `8883` in your firewall/router.
* Use `ufw` if on Ubuntu:

```bash
sudo ufw allow 1883/tcp
```

---

## 🛠 9. Troubleshooting Tips

| Issue                  | Solution                                               |
| ---------------------- | ------------------------------------------------------ |
| Broker not responding  | `systemctl status mosquitto`                           |
| Can’t connect remotely | Check firewall or bind address                         |
| Auth fails             | Re-check `/etc/mosquitto/passwd` and `allow_anonymous` |
| TLS errors             | Check cert paths and validity                          |

---

## 📘 10. Extras & Tools

* 🖥 **MQTT Dashboards**: [MQTT Explorer](https://mqtt-explorer.com/), [Node-RED](https://nodered.org/)
* 🛠 **Cloud MQTT brokers**: [HiveMQ Cloud](https://www.hivemq.com/mqtt-cloud-broker/), [Adafruit IO](https://io.adafruit.com/)
* 📦 **Docker Mosquitto**:

```bash
docker run -it -p 1883:1883 eclipse-mosquitto
```

---

## ✅ Summary Table

| Feature     | Default | Config File                       |
| ----------- | ------- | --------------------------------- |
| Port        | 1883    | `/etc/mosquitto/mosquitto.conf`   |
| Auth        | Off     | `/etc/mosquitto/conf.d/auth.conf` |
| TLS         | Off     | `/etc/mosquitto/conf.d/tls.conf`  |
| Persistence | Off     | `persistence true`                |
| Logs        | Syslog  | or `log_dest file`                |

## MQTT through python